In [1770]:
import numpy as np
import cv2
from statistics import median
import os
import matplotlib.pyplot as plt
from cv2 import BORDER_CONSTANT

In [1771]:
image = cv2.imread("output_image_crop_3.tif")
lw = [150,200,250,300,350,400,450,500,550,600]#A
#lw = [400,450,500,550,600]#B
lg = [200,225,250,275,300,325,350,375,400,425]
lg.reverse()
r = 10
c = 10
xborder = 494
yborder = 512
#xborder = 671
#yborder = 541
xsize = 950
ysize = 890
xbias = 0
ybias = -10
shift_th = 100
al_center = True
pth = "DatasetR1/R1new2-Gray-6400-right"
pth_rotate = "DatasetR1/R1new2-Gray-6400-right/rotated.jpg"
pth_bc = "DatasetR1/R1new2-Gray-6400-right/bc.jpg"
pth_resize = "DatasetR1/R1new2-Gray-6400-right/resized.jpg"
pth_split = "DatasetR1/R1new2-Gray-6400-right/split"
pth_split_name = 'DatasetR1/R1new2-Gray-6400-right/split/w{}_g{}_3.jpg'
pth_AEQ2 = "DatasetR1/R1new2-Gray-6400-right/splitAEQ2"
pth_AEQ2_name = 'DatasetR1/R1new2-Gray-6400-right/splitAEQ2/w{}_g{}_3.jpg'
pth_AEQ2norm = "DatasetR1/R1new2-Gray-6400-right/splitAEQ2norm"
pth_AEQ2norm_name = 'DatasetR1/R1new2-Gray-6400-right/splitAEQ2norm/w{}_g{}_3.jpg'

In [1772]:
def resize(img):
    scale_percent = 2000 * 100 / img.shape[0] # percent of original size
    #scale_percent = 2000 * 100 / img.shape[0] for the grayscale Dragonfly
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    print('scale:', scale_percent)
    return cv2.resize(img, dim, interpolation = cv2.INTER_AREA)



def rotate(img,angle,bg):
    cols = img.shape[1]
    rows = img.shape[0]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1) 

    return cv2.warpAffine(img,M,(cols,rows),borderValue=bg) 


def center_mass(img):
    #blur = cv2.GaussianBlur(img,(101,101),0)
    #ret3,image = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret3,image = cv2.threshold(img,80,255,cv2.THRESH_BINARY)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (10,10))
    mass_y, mass_x = np.where(image[:,:] >= 255)
    cent_y = np.average(mass_y)
    cent_x = np.average(mass_x)
    #print(cent_y,cent_x)
    return cent_y,cent_x

In [1773]:
# resize image
resized = resize(image)

image_gray = cv2.medianBlur(cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY),7)
#cv2.imwrite("gray.jpg",image_gray)

adaptive_threshold = cv2.adaptiveThreshold(image_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,1001,2)
#cv2.imwrite("at.jpg",adaptive_threshold)

edges = cv2.Canny(adaptive_threshold, 170, 220, apertureSize=3)
#cv2.imwrite("edge.jpg",edges)

theta_thres = np.pi/180 * 2
rho_thres = 40

for i in range(150,10**10,1):

    lines = cv2.HoughLines(edges, 1, np.pi / 1800, i, min_theta=np.pi-np.pi*5/180, max_theta=np.pi+np.pi*5/180)

    postlinesv = []
    for i in range(len(lines)):
        sim = False
        for line in postlinesv:
            rho1, theta1 = line[0]
            rho2, theta2 = lines[i][0]
            if abs(rho1-rho2) < rho_thres and abs(theta1-theta2) < theta_thres:
                sim = True
                break
        if not sim:
            postlinesv.append(lines[i])

    #print(len(postlinesv))
    if len(postlinesv) <= 50:
        break
    
for i in range(150,10**10,1):

    lines = cv2.HoughLines(edges, 1, np.pi / 1800, i, min_theta=np.pi/2-np.pi*5/180, max_theta=np.pi/2+np.pi*5/180)

    postlinesh = []
    for i in range(len(lines)):
        sim = False
        for line in postlinesh:
            rho1, theta1 = line[0]
            rho2, theta2 = lines[i][0]
            if abs(rho1-rho2) < rho_thres and abs(theta1-theta2) < theta_thres:
                sim = True
                break
        if not sim:
            postlinesh.append(lines[i])

    #print(len(postlines))
    if len(postlinesh) <= 50:
        break


for line in postlinesv+postlinesh:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 2000 * (-b))
    y1 = int(y0 + 2000 * (a))
    x2 = int(x0 - 2000 * (-b))
    y2 = int(y0 - 2000 * (a))
    cv2.line(resized, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
    

thetasv = [line[0][1] for line in postlinesv]
thetash = [line[0][1]+np.pi/2 for line in postlinesh]
mid_thetas = median(thetasv+thetash) - np.pi
angle = mid_thetas*180/np.pi
print(angle)

bg = np.average(image[:10,:10], axis = (0,1))
rot = rotate(image, angle,bg)

#plt.show()


if not os.path.exists(pth):
	os.mkdir(pth)

cv2.imwrite(pth_rotate,rot)

scale: 20.42275094455223
0.399998872841624


True

In [1774]:
def shift(img,x,y,bg):
    cols = img.shape[1]
    rows = img.shape[0]
    M = np.float32([[1,0,y],[0,1,x]])
    biased = cv2.warpAffine(img,M,(cols,rows))

    return cv2.warpAffine(img,M,(cols,rows),borderValue=bg) 



In [1775]:
image = cv2.imread(pth_rotate)

  
# resize image
resized = resize(image)
print(image.shape)

image_gray = cv2.medianBlur(cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY),7)

adaptive_threshold = cv2.adaptiveThreshold(image_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,1201,2)

edges = cv2.Canny(adaptive_threshold, 170, 220, apertureSize=3)

theta_thres = np.pi/180 * 2
rho_thres = 40

for i in range(150,10**10,1):

    lines = cv2.HoughLines(edges, 1, np.pi / 1800, i, min_theta=np.pi-np.pi*5/180, max_theta=np.pi+np.pi*5/180)

    postlinesv = []
    for i in range(len(lines)):
        sim = False
        for line in postlinesv:
            rho1, theta1 = line[0]
            rho2, theta2 = lines[i][0]
            if abs(rho1-rho2) < rho_thres and abs(theta1-theta2) < theta_thres:
                sim = True
                break
        if not sim:
            postlinesv.append(lines[i])

    #print(len(postlinesv))
    if len(postlinesv) <= shift_th:
        break
    
for i in range(150,10**10,1):

    lines = cv2.HoughLines(edges, 1, np.pi / 1800, i, min_theta=np.pi/2-np.pi*5/180, max_theta=np.pi/2+np.pi*5/180)

    postlinesh = []
    for i in range(len(lines)):
        sim = False
        for line in postlinesh:
            rho1, theta1 = line[0]
            rho2, theta2 = lines[i][0]
            if abs(rho1-rho2) < rho_thres and abs(theta1-theta2) < theta_thres:
                sim = True
                break
        if not sim:
            postlinesh.append(lines[i])

    #print(len(postlines))
    if len(postlinesh) <= shift_th:
        break


#maxh
min_idx = min(range(len(postlinesh)), key=lambda i: postlinesh[i][0][0])
rho, theta = postlinesh[min_idx][0]
a = np.cos(theta)
b = np.sin(theta)
x0 = a * rho
y0 = b * rho
x1 = int(x0 + 2000 * (-b))
y1 = int(y0 + 2000 * (a))
x2 = int(x0 - 2000 * (-b))
y2 = int(y0 - 2000 * (a))
cv2.line(resized, (x1, y1), (x2, y2), (0, 0, 255), thickness=5)

#minh
max_idx = max(range(len(postlinesh)), key=lambda i: postlinesh[i][0][0])
rho, theta = postlinesh[max_idx][0]
a = np.cos(theta)
b = np.sin(theta)
x0 = a * rho
y0 = b * rho
x1 = int(x0 + 2000 * (-b))
y1 = int(y0 + 2000 * (a))
x2 = int(x0 - 2000 * (-b))
y2 = int(y0 - 2000 * (a))
cv2.line(resized, (x1, y1), (x2, y2), (0, 0, 255), thickness=5)

#maxv
min_idx = min(range(len(postlinesv)), key=lambda i: postlinesv[i][0][0])
rho, theta = postlinesv[min_idx][0]
a = np.cos(theta)
b = np.sin(theta)
x0 = a * rho
y0 = b * rho
x1 = int(x0 + 2000 * (-b))
y1 = int(y0 + 2000 * (a))
x2 = int(x0 - 2000 * (-b))
y2 = int(y0 - 2000 * (a))
cv2.line(resized, (x1, y1), (x2, y2), (0, 0, 255), thickness=5)

#minv
max_idx = max(range(len(postlinesv)), key=lambda i: postlinesv[i][0][0])
rho, theta = postlinesv[max_idx][0]
a = np.cos(theta)
b = np.sin(theta)
x0 = a * rho
y0 = b * rho
x1 = int(x0 + 2000 * (-b))
y1 = int(y0 + 2000 * (a))
x2 = int(x0 - 2000 * (-b))
y2 = int(y0 - 2000 * (a))
cv2.line(resized, (x1, y1), (x2, y2), (0, 0, 255), thickness=5)


rhov = [line[0][0] for line in postlinesv]
rhoh = [line[0][0] for line in postlinesh]
scale_percent = 2000 / image.shape[0]
print(max(rhov) + min(rhov))
print(edges.shape[1])
biasv = ((max(rhov) + min(rhov)) + edges.shape[1]) / 2 / scale_percent
biash = -((max(rhoh) + min(rhoh)) - edges.shape[0]) / 2 / scale_percent
print(biash,biasv)

bg = np.average(image[:10,:10], axis = (0,1))
biased = shift(image,biash,biasv,bg)

#plt.imshow(resized)
#plt.show()

cv2.imwrite(pth_bc,biased)



scale: 20.42275094455223
(9793, 11805, 3)
-2350.0
2410
70.99925 146.895


True

In [1776]:
biased = cv2.imread(pth_bc)
h,w,_ = biased.shape
vert=0
hori=0
y_tune=9000
x_tune=10000
#1y = 9000
#1x = 10000
#2y = 9000
#2x = 9950
#3y = 9000
#3x = 9950
x_moving_center=740
y_moving_center=203
#1x = 600
#2x = 740
#3x = 740
#result 1 y moving center = 250
#result 2 y moving center = -30
#result 3 y moving center = 120
if h < y_tune:
    vert = y_tune - h
if w < x_tune:
    hori = x_tune - w
print(biased[0][0])
resized = cv2.copyMakeBorder(biased, vert//2, vert//2, hori//2, hori//2, BORDER_CONSTANT, None,biased[0][0].tolist())

center = resized.shape
x = center[1]/2 - x_tune/2 + x_moving_center
y = center[0]/2 - y_tune/2 + y_moving_center

resized = resized[int(y):int(y+y_tune), int(x):int(x+x_tune)]
cv2.imwrite(pth_resize,resized)

[53 53 53]


True

In [1789]:


#border is the distance from the border to the center of item closest to the border 
def split(img, r, c, xborder, yborder, xsize, ysize, xbias, ybias):
    h,w,_ = img.shape
    xdis = 1003
    #xdis=1000
    ydis = 887.5
    #ydis=885 or 886.3
    #print(ydis)
    x_shift_base = 13
    #bas = 13.1
    for i in range(c):
        for j in range(r):
            x_shift = -int(i * x_shift_base)
            if j in [5,6,7,8,9]:
              x_shift += 7
            if j in [2,3,4]:
              x_shift += 4
            if i in [5,6, 7, 8, 9]:
              x_shift -= 3
            if i in [6, 7, 8, 9]:
              x_shift -= 4
            if i in [7, 8, 9]:
              x_shift -= 2
            if i in [8, 9]:
              x_shift -= 5
            if i == 9:
              x_shift -= 5
            
            print(f"i = {i}, j = {j}, x_shift = {x_shift}")
            x_min = int(xborder + i * xdis - xsize / 2 + xbias )
            x_max = int(xborder + i * xdis + xsize / 2 + xbias )
            y_min = int(yborder + j * ydis - ysize / 2 + ybias )
            y_max = int(yborder + j * ydis + ysize / 2 + ybias )
            
            crop_img = img[y_min:y_max, x_min:x_max]
            grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            norm_image = (cv2.normalize(grey, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) * 255).astype(np.uint8)
            #print("crop_img:", y_min, y_max, x_min, x_max)
            if al_center:
                center_y, center_x = center_mass(norm_image)
                shift_y = 0
                if i in [4,5,6,7,8,9]:
                   shift_y = 8
                #print(shift_y)
                shift_x = 0
                crop_img = img[y_min + shift_y:y_max + shift_y, x_min + shift_x+x_shift:x_max + shift_x+x_shift]
                #print("cropimage: ", y_min + shift_y, y_max + shift_y, x_min + shift_x, x_max + shift_x)
                grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            
            if not os.path.exists(pth_split):
                os.mkdir(pth_split)
            
            cv2.imwrite(pth_split_name.format(lw[i], lg[j]), grey)
            clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(50, 50))
            cl1 = clahe.apply(grey)
            
            if not os.path.exists(pth_AEQ2):
                os.mkdir(pth_AEQ2)
            
            cv2.imwrite(pth_AEQ2_name.format(lw[i], lg[j]), cl1)
            norm_image = (cv2.normalize(cl1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) * 255).astype(np.uint8)
            
            if not os.path.exists(pth_AEQ2norm):
                os.mkdir(pth_AEQ2norm)
            
            cv2.imwrite(pth_AEQ2norm_name.format(lw[i], lg[j]), norm_image)


image = cv2.imread(pth_resize)
#cv2.imshow('1',image)
split(image, r, c, xborder, yborder, xsize, ysize, xbias, ybias)
            

i = 0, j = 0, x_shift = 0
i = 0, j = 1, x_shift = 0
i = 0, j = 2, x_shift = 4
i = 0, j = 3, x_shift = 4
i = 0, j = 4, x_shift = 4
i = 0, j = 5, x_shift = 7
i = 0, j = 6, x_shift = 7
i = 0, j = 7, x_shift = 7
i = 0, j = 8, x_shift = 7
i = 0, j = 9, x_shift = 7
i = 1, j = 0, x_shift = -13
i = 1, j = 1, x_shift = -13
i = 1, j = 2, x_shift = -9
i = 1, j = 3, x_shift = -9
i = 1, j = 4, x_shift = -9
i = 1, j = 5, x_shift = -6
i = 1, j = 6, x_shift = -6
i = 1, j = 7, x_shift = -6
i = 1, j = 8, x_shift = -6
i = 1, j = 9, x_shift = -6
i = 2, j = 0, x_shift = -26
i = 2, j = 1, x_shift = -26
i = 2, j = 2, x_shift = -22
i = 2, j = 3, x_shift = -22
i = 2, j = 4, x_shift = -22
i = 2, j = 5, x_shift = -19
i = 2, j = 6, x_shift = -19
i = 2, j = 7, x_shift = -19
i = 2, j = 8, x_shift = -19
i = 2, j = 9, x_shift = -19
i = 3, j = 0, x_shift = -39
i = 3, j = 1, x_shift = -39
i = 3, j = 2, x_shift = -35
i = 3, j = 3, x_shift = -35
i = 3, j = 4, x_shift = -35
i = 3, j = 5, x_shift = -32
i = 3, j = 6, x_shif